背景：
    整理单目录下10W+的照片文件，名称YYYYMMDD_IMG_xxxx.(jpg\png\heic)
需求：
    1.将jpg\png以外格式的文件，移入other
    2.根据文件名YYYYMMDD，移入对应日期的文件夹(例如:202107)
    3.去除文件名中的日期(YYYYMMDD_)

In [ ]:
import os, shutil

PATH = r"\\192.168.31.99\photo\hanminmin\相机胶卷"
FILE_LIST = ['.JPG','.PNG']

def check_create_dir(dir_path):
    fd = os.path.exists(dir_path)
    if not fd:
        os.makedirs(dir_path)

def move_file(file_path, dir_path):
    check_create_dir(dir_path)
    shutil.move(file_path, dir_path)

def do_file(file, current_dir_path):
    file_path = os.path.join(current_dir_path, file)
    file_split = os.path.splitext(file)
    file_type = file_split[1]
    if(file_type in FILE_LIST):
        file_name = str(file_split[0])
        date_dir_name = file_name.split('_')[0]
        date_dir_path = os.path.join(current_dir_path, date_dir_name[0:6])
        print('检查创建文件夹:'+date_dir_path)
        check_create_dir(date_dir_path)
        move_file(file_path, date_dir_path)
    else:
        other_dir_path = os.path.join(current_dir_path, 'other')
        move_file(file_path, other_dir_path)


第一步
将指定文件夹下的文件整理到对应的日期文件夹下

In [ ]:
def move_my_file(my_file_path):
    for root, dirs, files in os.walk(my_file_path):
        print('root:')
        print(root)  # 当前目录路径
        print('dirs:')
        print(dirs)  # 当前路径下所有子目录
        print('files:')
        # print(files)  # 当前路径下所有非目录子文件
        print('========================')
        for file in files:
            do_file(file, root)
    print('success')

# move_my_file(PATH)

第二步
遍历所有文件并修改文件名(去除日期YYYYMMDD_开头)

In [ ]:
def rename_my_file(my_file_path):
    for root, dirs, files in os.walk(my_file_path):
        print('root:')
        print(root)  # 当前目录路径
        print('dirs:')
        print(dirs)  # 当前路径下所有子目录
        print('files:')
        # print(files)  # 当前路径下所有非目录子文件
        print('========================')
        for file in files:
            if file[0:3] == 'IMG':
                break
            else:
                file_path = os.path.join(root, file)
                print('file_path:'+file_path)
                new_file_name = file[9:]
                print('new_file_name:'+new_file_name)
                new_file_path = os.path.join(root, new_file_name)
                os.rename(file_path, new_file_path)
    print('success')

# rename_my_file(PATH)

In [12]:
my_file_path = r"\\192.168.31.99\photo\hanminmin\photo\相机胶卷"
move_my_file(my_file_path)
# rename_my_file(my_file_path)

NameError: name 'move_my_file' is not defined